# 416 — Partition Equal Subset Sum

Given an integer array nums, return true if you can partition the array into two subsets such that the sum of the elements in both subsets is equal or false otherwise.


### Example 1:

- Input: **nums = [1,5,11,5]**
- Output: true
- Explanation: The array can be partitioned as [1, 5, 5] and [11].

### Example 2:

- Input: **nums = [1,2,3,5]**
- Output: false
- Explanation: The array cannot be partitioned into equal sum subsets.


# Base Version

In [4]:
from typing import List

def canPartition(nums: List[int]) -> bool:
    total = sum(nums)
    if total % 2:
        return False
    target = total // 2

    dp = [False] * (target + 1)
    dp[0] = True

    for x in nums:
        for s in range(target, x - 1, -1):
            if dp[s - x]:
                dp[s] = True
        if dp[target]:
            return True

    return dp[target]

canPartition([1, 5, 11, 5])

True

# Verbose Version

In [5]:
from typing import List, Iterable

def _summarize_list(vals: Iterable[int], max_items: int = 30) -> str:
    """Pretty-print a (possibly long) iterable of ints."""
    vals = list(vals)
    n = len(vals)
    if n <= max_items:
        return str(vals)
    half = max_items // 2
    head = ", ".join(map(str, vals[:half]))
    tail = ", ".join(map(str, vals[-half:]))
    return f"[{head}, …, {tail}] (total {n})"

def _print_dp_snapshot(dp: List[bool], target: int, max_items: int = 30) -> None:
    """Show which sums are reachable so far (with truncation on large cases)."""
    sums = [i for i, ok in enumerate(dp) if ok]
    print("    reachable sums:", _summarize_list(sums, max_items))

def canPartition_verbose(nums: List[int], *, max_items_to_show: int = 30) -> bool:
    """
    Verbose solver for LeetCode 416 (0/1 subset sum to target = total//2).
    Prints the DP state changes after processing each item.

    Args:
        nums: list of positive ints.
        max_items_to_show: how many entries to show in summaries (to avoid huge logs).

    Returns:
        bool: True if a partition exists, else False.
    """
    print("=== Partition Equal Subset Sum — Verbose Trace ===")
    print(f"Input nums: {nums}")

    total = sum(nums)
    print(f"Total sum = {total}")

    if total % 2 == 1:
        print("Total is odd -> cannot be split into two equal subsets. Answer: False")
        return False

    target = total // 2
    print(f"Target subset sum = total//2 = {target}")

    # Quick pruning: if any number exceeds target, impossible.
    mx = max(nums) if nums else 0
    print(f"Max element = {mx}")
    if mx > target:
        print(f"An element ({mx}) > target ({target}) -> Answer: False")
        return False

    # dp[s] == True  <=>  some subset achieves sum s
    dp = [False] * (target + 1)
    dp[0] = True
    print("Initial DP:")
    _print_dp_snapshot(dp, target, max_items_to_show)

    # Iterate items (0/1 choice) and update sums backwards
    for idx, x in enumerate(nums):
        print(f"\n-- Item {idx} | value = {x} --")
        changed = []   # which sums newly became reachable due to x
        # Backward loop enforces 0/1 usage (no reuse of same item)
        for s in range(target, x - 1, -1):
            if dp[s - x] and not dp[s]:
                dp[s] = True
                changed.append(s)

        if changed:
            print("  Newly reachable sums with this item:",
                  _summarize_list(sorted(changed), max_items_to_show))
        else:
            print("  No new sums became reachable with this item.")

        # Show current snapshot (truncated if large)
        print("  DP after this item:")
        _print_dp_snapshot(dp, target, max_items_to_show)

        if dp[target]:
            print(f"Target {target} reached. Early exit -> Answer: True")
            return True

    ans = dp[target]
    print(f"\nFinished processing all items. Answer: {ans}")
    return ans


# --- Example runs (you can comment these out on LeetCode) ---

if __name__ == "__main__":
    print("\nExample 1:")
    canPartition_verbose([1, 5, 11, 5], max_items_to_show=20)

    # print("\nExample 2:")
    # canPartition_verbose([1, 2, 3, 5], max_items_to_show=20)



Example 1:
=== Partition Equal Subset Sum — Verbose Trace ===
Input nums: [1, 5, 11, 5]
Total sum = 22
Target subset sum = total//2 = 11
Max element = 11
Initial DP:
    reachable sums: [0]

-- Item 0 | value = 1 --
  Newly reachable sums with this item: [1]
  DP after this item:
    reachable sums: [0, 1]

-- Item 1 | value = 5 --
  Newly reachable sums with this item: [5, 6]
  DP after this item:
    reachable sums: [0, 1, 5, 6]

-- Item 2 | value = 11 --
  Newly reachable sums with this item: [11]
  DP after this item:
    reachable sums: [0, 1, 5, 6, 11]
Target 11 reached. Early exit -> Answer: True


# What dp[s] Actually Means

- dp[s] == True means:

"Using some subset of the numbers we’ve seen so far, we can make sum s."

---

- So after processing all numbers, if dp[11] is True, it literally means:

"There exists a subset of nums whose sum = 11."

---

And that’s exactly what we want — a subset summing to half of total (11 in this case).

If one subset sums to 11, the rest of the numbers automatically sum to total - 11 = 11.
So we successfully split the array into two equal parts.